In [1]:
%matplotlib inline

In [2]:
import requests
import re
from fastai.text import *
import torch

In [3]:
book = requests.get('https://www.gutenberg.org/files/36/36-0.txt').text

In [4]:
start_iter = re.finditer('BOOK ONE', book)

In [5]:
next(start_iter)

<_sre.SRE_Match object; span=(1067, 1075), match='BOOK ONE'>

Commenting out some print lines here to shrink the notebook -- I did some exploration to figure out where the actual story began and ended (excluding table of contents, acknowledgements, etc.)

In [6]:
# book[1067:]

In [7]:
next(start_iter)

<_sre.SRE_Match object; span=(1920, 1928), match='BOOK ONE'>

In [8]:
# book[1920:]

In [9]:
book = book[1920:]

In [10]:
end_iter = re.finditer("among the dead.", book)

In [11]:
next(end_iter)

<_sre.SRE_Match object; span=(64109, 64124), match='among the dead '>

In [12]:
# book[:64125]

In [13]:
next(end_iter)

<_sre.SRE_Match object; span=(345594, 345609), match='among the dead.'>

In [14]:
# book[:345609]

In [15]:
book = book[:345609]

In [16]:
book = re.sub(r'[\r\n]+', ' ', str(book))

In [17]:
# book

In [18]:
tokens = [i.replace(',', '') for i in book.split('.')]

In [19]:
# tokens

In [20]:
tokens[:10]

['BOOK ONE THE COMING OF THE MARTIANS I',
 ' THE EVE OF THE WAR',
 ' No one would have believed in the last years of the nineteenth century that this world was being watched keenly and closely by intelligences greater than manâ\x80\x99s and yet as mortal as his own; that as men busied themselves about their various concerns they were scrutinised and studied perhaps almost as narrowly as a man with a microscope might scrutinise the transient creatures that swarm and multiply in a drop of water',
 ' With infinite complacency men went to and fro over this globe about their little affairs serene in their assurance of their empire over matter',
 ' It is possible that the infusoria under the microscope do the same',
 ' No one gave a thought to the older worlds of space as sources of human danger or thought of them only to dismiss the idea of life upon them as impossible or improbable',
 ' It is curious to recall some of the mental habits of those departed days',
 ' At most terrestrial men fa

In [21]:
! rm book_text.csv

In [22]:
with open('book_text.csv', 'a+') as file:
    for sentence in tokens:
        file.write(sentence + '\n')

In [23]:
with open('book_text.csv') as file:
    lines = file.readlines()[:5]
print(lines[0])
print(lines[1])
print(lines[2])

BOOK ONE THE COMING OF THE MARTIANS I

 THE EVE OF THE WAR

 No one would have believed in the last years of the nineteenth century that this world was being watched keenly and closely by intelligences greater than manâs and yet as mortal as his own; that as men busied themselves about their various concerns they were scrutinised and studied perhaps almost as narrowly as a man with a microscope might scrutinise the transient creatures that swarm and multiply in a drop of water



In [24]:
pd.read_csv('book_text.csv').iloc[20][0]

' And looking across space with instruments and intelligences such as we have scarcely dreamed of they see at its nearest distance only 35000000 of miles sunward of them a morning star of hope our own warmer planet green with vegetation and grey with water with a cloudy atmosphere eloquent of fertility with glimpses through its drifting cloud wisps of broad stretches of populous country and narrow navy-crowded seas'

In [25]:
data = TextLMDataBunch.from_csv('.', 'book_text.csv', text_cols=0)

In [26]:
data

TextLMDataBunch;

Train: LabelList (2434 items)
x: LMTextList
xxbos xxmaj in one night the valley had become a valley of xxunk,xxbos xxmaj hidden by a pine wood as they were they seem to have been quite unsuspected by the xxmaj martian nearest to them,xxbos â âhow do you xxunk said the artilleryman,xxbos xxmaj the night was unexpectedly dark ; to me walking out of the lighted passage of my cousinsâ house it seemed indeed black and it was as hot and close as the day,xxbos âpoor xxmaj xxunk i said
y: LMLabelList
,,,,
Path: .;

Valid: LabelList (609 items)
x: LMTextList
xxbos â xxunk xxup story xxup from xxunk and so forth,xxbos xxmaj but though i gnawed parts of these in my mouth there was nothing to be got from them,xxbos xxmaj it was just as well for them for the mere attempt to stand xxunk upon our planet would have broken every xxunk in their bodies,xxbos xxmaj this was the xxmaj xxunk,xxbos xxmaj that brought him to reason for a time
y: LMLabelList
,,,,
Path: .;

Test: Non

In [27]:
data.train_ds[0]

(Text xxbos xxmaj in one night the valley had become a valley of xxunk,
 EmptyLabel )

In [28]:
data.train_ds[1][0].data

array([   2,    5,  381,   33,   12,  243,  623,   23,   30,   26,   30,  259,   14,   39,   50,  234, 2045,   33,
          9,    5,  103, 1239,   14,   51])

In [29]:
data.valid_ds[1][0].data

array([   2,    5,   31,  291,   13, 1283, 1841,   11,   66,   15,   20,  840,   38,   16,  163,   14,   68,  225,
         28,   51])

In [30]:
data.one_batch()[0].shape

torch.Size([64, 70])

In [31]:
x, y = data.one_batch()

In [32]:
x.shape, y.shape

(torch.Size([64, 70]), torch.Size([64, 70]))

In [33]:
x[0], y[0]

(tensor([  34,   10,  108,   21,  115,    9,  680, 2051,    9, 1240,  410,   11,
           69,  337, 2052,   10,  163,   11,    9,  106,    5,  868,    5,  626,
          106,   73,   68,  122,   40,  627,   73,    9,  319,    5,  103,   68,
          116,    2,    5,   18,   16,   64,    0,    0,  156,   11,    0,  220,
           10,   52,   15,   12,  338,   88,  767,    9,  102, 1021,   11,    9,
            5, 2053,  147,    9,  681,   11,    9,  151,    9,  178]),
 tensor([  10,  108,   21,  115,    9,  680, 2051,    9, 1240,  410,   11,   69,
          337, 2052,   10,  163,   11,    9,  106,    5,  868,    5,  626,  106,
           73,   68,  122,   40,  627,   73,    9,  319,    5,  103,   68,  116,
            2,    5,   18,   16,   64,    0,    0,  156,   11,    0,  220,   10,
           52,   15,   12,  338,   88,  767,    9,  102, 1021,   11,    9,    5,
         2053,  147,    9,  681,   11,    9,  151,    9,  178,  682]))

In [34]:
len(data.vocab.itos)

3128

In [35]:
nv = len(data.vocab.itos)

In [36]:
nh = 64

In [37]:
bs = 64

In [38]:
nv, nh, bs

(3128, 64, 64)

In [41]:
class BasicLanguageModel(nn.Module):
     
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv, nh)
        self.h_h = nn.Linear(nh, nh)
        self.h_o = nn.Linear(nh, nv)
        self.bn = nn.BatchNorm1d(nh)
        self.reset()
        
    def forward(self, x):
        print("input size: ", x.size())
        res = []
        h = self.h
        for i in range(x.shape[1]):
            h = h + self.i_h(x[:,i]) 
            h = F.relu(self.h_h(h))
            res.append(self.bn(h))
        print("hidden layer size: ", h.size())
        print("res size: ", res[0].size())
        self.h = h.detach()
        res = torch.stack(res, dim=1)
        print("stacked res size: ", res.size())
        print("output size: ", self.h_o(res).size())
        return self.h_o(res)
    
    def reset(self):
        self.h = torch.zeros(bs, nh).cuda()

In [47]:
learn = LanguageLearner(data, BasicLanguageModel(), metrics=accuracy)

In [46]:
learn.summary()

input size:  torch.Size([1, 70])
hidden layer size:  torch.Size([64, 64])
res size:  torch.Size([64, 64])
stacked res size:  torch.Size([64, 70, 64])
output size:  torch.Size([64, 70, 3128])


BasicLanguageModel
Layer (type)         Output Shape         Param #    Trainable 
Embedding            [64]                 200,192    True      
______________________________________________________________________
Linear               [64]                 4,160      True      
______________________________________________________________________
Linear               [70, 3128]           203,320    True      
______________________________________________________________________
BatchNorm1d          [64]                 128        True      
______________________________________________________________________

Total params: 407,800
Total trainable params: 407,800
Total non-trainable params: 0
Optimized with 'torch.optim.adam.Adam', betas=(0.9, 0.99)
Using true weight decay as discussed in https://www.fast.ai/2018/07/02/adam-weight-decay/ 
Loss function : FlattenedLoss
Callbacks functions applied 
    RNNTrainer

In [122]:
data.show_batch()

idx,text
0,sailors could no longer come up the xxmaj thames they came on to the xxmaj essex coast to xxmaj harwich and xxmaj walton and xxmaj xxunk and afterwards to xxmaj foulness and xxmaj xxunk to bring off the people xxbos â i resolved to leave xxunk that i had ! xxmaj xxunk now for the xxunk xxunk i xxunk out food and drink xxbos xxmaj haggard special xxunk with white
1,way!â xxmaj it was like riding into the smoke of a fire to approach the meeting point of the lane and road ; the crowd xxunk like a fire and the dust was hot and pungent xxbos âulla ulla ulla ullaâ cried the voice coming as it seemed to me from the district about xxmaj regentâs xxmaj park xxbos xxmaj its physical condition is still xxunk a xxunk but we
2,two days of xxunk and rain and at xxmaj clapham xxmaj junction the line had been wrecked again ; there were hundreds of out - of - work clerks and shopmen working side by side with the xxunk navvies and we were jolted over a hasty xxunk xxbos â âwhat xxunk xxunk xxunk âno xxunk and the artilleryman began a vivid account of the xxmaj heat - xxmaj ray xxbos
3,curate and stopped at the scullery door xxbos i whispered for the curate several times and at last felt my way to the door of the kitchen xxbos xxmaj we got to xxmaj hampton xxmaj court without misadventure our minds full of strange and unfamiliar appearances and at xxmaj hampton xxmaj court our eyes were relieved to find a patch of green that had escaped the xxunk xxunk xxbos i
4,road xxbos xxmaj iâve been in sight of death once or twice ; xxmaj iâm not an xxunk soldier and at the best and worst xxunk just death xxbos xxmaj that xxunk up xxbos xxmaj the vapour did not xxunk as a true gas would do xxbos xxmaj close on the rear of this came a couple of cabs the xxunk of a long xxunk of flying vehicles going for


In [123]:
learn.data.bs

64

In [124]:
learn.fit_one_cycle(10, max_lr=3e-2)

epoch,train_loss,valid_loss,accuracy,time


input size:  torch.Size([64, 70])
hidden layer size:  torch.Size([64, 64])
res size:  torch.Size([64, 64])
stacked res size:  torch.Size([64, 70, 64])
output size:  torch.Size([64, 70, 3120])


ValueError: Expected input batch_size (70) to match target batch_size (4480).